In [1]:
from TeamModule import Team, teamBuilder
import pandas as pd

In [14]:
teamLookups = {
    'AE' : {'division' : 'East', 
            'name' : 'Austin Eck'},
    'BC' : {'division' : 'West', 
            'name' : 'Brent Chapman'},
    'CB' : {'division' : 'West', 
            'name' : 'Cody Blakeney'},
    'JE' : {'division' : 'East', 
            'name' : 'Jim Eck'},
    'JR' : {'division' : 'East', 
            'name' : 'Jason Radich'},
    'MH' : {'division' : 'East', 
            'name' : 'Mike Hern'},
    'MW' : {'division' : 'West', 
            'name' : 'Matt Ward'},
    'PDO' : {'division' : 'West', 
            'name' : 'Philip DeOchoa'},
    'RS' : {'division' : 'East', 
            'name' : 'Ryan Stange'},
    'SB' : {'division' : 'West', 
            'name' : 'Stephen Baker'},
}
teams = [teamBuilder(teamLookups[team]['name'], teamLookups[team]['division'], team+'schedule.txt') for team in teamLookups]

teamDF = pd.DataFrame(data = {'Name' : [team.owner for team in teams], 
                          'Division' : [team.division for team in teams], 
                         'Wins' : [team.wins for team in teams], 
                         'Losses' : [team.losses for team in teams], 
                         })

scoresDF = pd.DataFrame(data = {team.owner : team.scores for team in teams}).T
scoresDF.columns = ['Week '+str(x) for x in range(1,len(scoresDF.columns)+1)]
scoresDF

master_df = pd.merge(teamDF, scoresDF, left_on='Name', right_index=True, how = 'inner')
master_df

,Name,Division,Wins,Losses,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8
0,Austin Eck,East,6,2,145.36,171.50,113.34,109.78,145.42,144.30,100.64,143.62
1,Brent Chapman,West,1,7,125.52,121.74,132.74,116.52,104.22,127.88,93.30,126.98
2,Cody Blakeney,West,6,2,167.96,171.30,152.24,80.72,134.26,164.16,137.44,88.56
3,Jim Eck,East,5,3,88.92,159.80,134.04,168.32,166.76,149.70,141.66,103.66
4,Jason Radich,East,1,7,151.72,68.88,88.92,131.72,104.18,67.40,90.80,118.10
5,Mike Hern,East,4,4,130.72,142.42,123.20,156.02,145.78,133.48,121.24,100.40
6,Matt Ward,West,4,4,153.68,104.22,136.14,139.08,112.42,179.60,76.64,142.12
7,Philip DeOchoa,West,5,3,84.70,125.96,158.42,107.32,200.90,87.32,161.04,115.76
8,Ryan Stange,East,5,3,122.56,138.12,131.02,128.56,138.88,129.88,135.28,73.66
9,Stephen Baker,West,3,5,122.00,120.46,124.78,119.44,160.68,87.08,134.88,71.56


In [38]:
df = master_df.copy() 

df = df[[x for x in df.columns if x == 'Name' or 'Week' in x]]
df.set_index('Name', inplace = True)
s = df.rank().apply(lambda x: x/10).sum(axis = 1).rename('xW')

In [47]:
results = master_df.copy()
results = pd.merge(results, s, right_index = True, left_on='Name')
results = results[['Name', 'Wins', 'Losses', 'xW']]
results['LuckyWins'] = results.Wins - results.xW
results.sort_values('LuckyWins', ascending = False)

,Name,Wins,Losses,xW,LuckyWins
0,Austin Eck,6,2,4.9,1.1
8,Ryan Stange,5,3,4.0,1.0
2,Cody Blakeney,6,2,5.3,0.7
7,Philip DeOchoa,5,3,4.7,0.3
9,Stephen Baker,3,5,3.2,-0.2
5,Mike Hern,4,4,4.7,-0.7
3,Jim Eck,5,3,5.8,-0.8
6,Matt Ward,4,4,5.0,-1.0
4,Jason Radich,1,7,2.8,-1.8
1,Brent Chapman,1,7,3.6,-2.6
